In [ ]:
import pandas as pd
file_path = 'main_data.csv'
df = pd.read_csv(file_path)
print(df.shape)
df = df.drop(['production_companies', 'production_countries','genres'], axis=1)
print(df.shape)
df = df[df['year'] >= 1750]
print(df.shape)
df = df[~df.applymap(lambda x: x == 'Empty!').any(axis=1)]
print(df.shape)
df=df.drop_duplicates()
print(df.shape)


In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import csv
from sklearn.preprocessing import OneHotEncoder
import random
from sklearn.ensemble import RandomForestRegressor
from collections import Counter
import re
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression
import joblib
from scipy.sparse import hstack
from sklearn.preprocessing import LabelEncoder
import itertools
import time






def naiveB(test_x,train_x,train_y,test_y,name):


    clf = GaussianProcessRegressor()
    clf.fit(train_x, train_y)
    predict = clf.predict(test_x)

    mse = mean_squared_error(test_y, predict)
    mae = mean_absolute_error(test_y, predict)
    r2 = r2_score(test_y, predict)

    print('________________Результаты модели_________________')
    print('Mean Squared Error (MSE):', mse)
    print('Mean Absolute Error (MAE):', mae)
    print('R2 Score:', r2)
    joblib.dump(clf, name)  
    print(f'Model saved as {name}')


def LinearRegressionModel(test_x, train_x, train_y, test_y, name):
    best_model = None
    best_features = []
    best_mse = float('inf')

    for n_features in [12, 13]:
        if n_features <= len(train_x.columns): 
            for feature_set in itertools.combinations(train_x.columns, n_features):
                X_train_subset = train_x[list(feature_set)]
                X_test_subset = test_x[list(feature_set)]

                lr = LinearRegression()
                lr.fit(X_train_subset, train_y)

                predict = lr.predict(X_test_subset)

                mse = mean_squared_error(test_y, predict)
                mae = mean_absolute_error(test_y, predict)
                r2 = r2_score(test_y, predict)

                print(f"Testing features: {feature_set}")
                print(f"Mean Squared Error: {mse}")
                print(f"Mean Absolute Error: {mae}")
                print(f"R2 Score: {r2}")
                print("-" * 40)

                if mse < best_mse:
                    best_mse = mse
                    best_model = lr
                    best_features = feature_set

    joblib.dump(best_model, name)
    print(f"Best model saved with features {best_features} as {name}")
    return best_features

def SVR_alg(test_x, train_x, train_y, test_y, name):
    param_grid = {
        'kernel': ['poly'],
        'degree': [2,3],
        'C': [10],
        'coef0': [1]
    }
    grid_search = GridSearchCV(SVR(), param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(train_x, train_y)

    best_params = grid_search.best_params_
    print(f'Best parameters: {best_params}')

    clf = SVR(
        kernel=best_params['kernel'],
        degree=best_params['degree'],
        C=best_params['C'],
        coef0=best_params['coef0']
    )

    clf.fit(train_x, train_y)

    predict = clf.predict(test_x)

    mse = mean_squared_error(test_y, predict)
    mae = mean_absolute_error(test_y, predict)
    r2 = r2_score(test_y, predict)

    print('________________Результаты модели_________________')
    print('Mean Squared Error (MSE):', mse)
    print('Mean Absolute Error (MAE):', mae)
    print('R2 Score:', r2)
    print('________________Опорные вектора_________________')
    support_vectors = clf.support_
    print('Количество опорных векторов:', len(support_vectors))

    joblib.dump(clf, name)  
    print(f'Model saved as {name}')


def KNN_alg(test_x, train_x, train_y, test_y,name):

    param_grid = {'n_neighbors': np.arange(10, 130, 30)}  
    grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(train_x, train_y)

    best_n_neighbors = grid_search.best_params_['n_neighbors']
    print(f'Best number of neighbors: {best_n_neighbors}')

    knn = KNeighborsRegressor(n_neighbors=best_n_neighbors)
    knn.fit(train_x, train_y)

    predict = knn.predict(test_x)

    mse = mean_squared_error(test_y, predict)
    mae = mean_absolute_error(test_y, predict)
    r2 = r2_score(test_y, predict)

    print('Mean Squared Error:', mse)
    print('Mean Absolute Error:', mae)
    print('R2 Score:', r2)
    joblib.dump(knn, name)  
    print(f'Model saved as {name}')

def RandomForest(test_x, train_x, train_y, test_y, name):
    param_grid = {'n_estimators': np.arange(100, 400, 50)}
    grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(train_x, train_y)

    best_n_estimators = grid_search.best_params_['n_estimators']
    print(f'Best number of estimators: {best_n_estimators}')

    rfr = RandomForestRegressor(n_estimators=best_n_estimators, random_state=42)
    rfr.fit(train_x, train_y)

    predict = rfr.predict(test_x)

    mse = mean_squared_error(test_y, predict)
    mae = mean_absolute_error(test_y, predict)
    r2 = r2_score(test_y, predict)

    print('Random Forest - Mean Squared Error:', mse)
    print('Random Forest - Mean Absolute Error:', mae)
    print('Random Forest - R2 Score:', r2)

    joblib.dump(rfr, name) 
    print(f'Model saved as {name}')
label_encoder = LabelEncoder()
df['status'] = label_encoder.fit_transform(df['status'])

label_encoder = LabelEncoder()
df['month_name'] = label_encoder.fit_transform(df['month_name'])

label_encoder = LabelEncoder()
df['original_language'] = label_encoder.fit_transform(df['original_language'])

label_encoder = LabelEncoder()
df['adult'] = label_encoder.fit_transform(df['adult'])

# label_encoder = LabelEncoder()
# df['production_companies'] = label_encoder.fit_transform(df['month_name'])

# label_encoder = LabelEncoder()
# df['production_countries'] = label_encoder.fit_transform(df['original_language'])

# label_encoder = LabelEncoder()
# df['genres'] = label_encoder.fit_transform(df['adult'])

columns_to_keep = [
    'vote_average', 'vote_count', 'status', 'revenue', 'runtime', 
    'adult', 'budget', 'original_language', 'popularity',  'numVotes', 'year', 'month_name', 'population_sum'
]
# columns_to_keep = [
#     'vote_average', 'vote_count', 'status', 'revenue', 'runtime', 
#     'adult', 'budget', 'original_language', 'popularity',  'numVotes', 'year', 'month_name', 'population_sum'
# ]
# columns_to_keep = [
#     'vote_average', 'vote_count',  'revenue', 'runtime', 
#     'adult', 'budget',  'popularity',  'numVotes', 'year', 'month_name', 'population_sum'
# ]
X = df[columns_to_keep]
Y=df['averageRating']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


start_time = time.time()
RandomForest(test_x=X_test, train_x=X_train, train_y=Y_train, test_y=Y_test, name='forest_main_1.joblib')
print(f"RandomForest time: {time.time() - start_time} seconds")
print()
# SVR_alg

start_time = time.time()
naiveB(test_x=X_test, train_x=X_train, train_y=Y_train, test_y=Y_test, name='baies_main_1.joblib')
print(f"NaiveB time: {time.time() - start_time} seconds")
print()

